## dataLoad

In [154]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, post_slack
base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함

## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']

In [163]:
train, valid = custom_train_test_split(train_df, ratio=0.7, seed=13) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((2525956, 26), (1974, 26), (260114, 26))

((2525956, 25), (2525956, 1), (1974, 25), (1974, 1))

In [164]:
def custom_train_test_split(df, ratio=0.7, seed=41, split=True):
    """df를 입력받아서, userid들을 랜덤해서 1-ratio만큼의 user만 선별하고, 마지막 제출껀만 만들어서 train, test로 리턴 """
    random.seed(seed)
    ## (user_id, 문제푼 횟수)를 원소로 갖는 리스트
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users) # 셔플
    max_train_data_len = ratio*len(df) # train data 길이
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users: # for문으로 전체의 ratio 퍼센트만 user_ids에 추가
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)

    ## train, test split
    train = df[df['userID'].isin(user_ids)] # train
    test = df[df['userID'].isin(user_ids) == False] # test

    #test데이터셋은, train에서 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [168]:
train, valid = custom_train_test_split(train_df, ratio=0.7, seed=13) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
#     train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((2525956, 25), (2525956, 1), (1974, 25), (1974, 1))

In [157]:
df[:2]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,month,...,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,kind,solvesec,solvesec_3600,time_category
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,NaN,0,NaN,3,...,0.947683,0.222749,1268,0.955022,0.207410,637,train,0.0,0.0,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,1,1.0,3,...,0.947683,0.222749,1268,0.913187,0.281603,3040,train,3.0,3.0,1


### 시드별 Validation set align

In [162]:
for seed in range(100):
    print(f"seed: {seed}")
    train, valid = custom_train_test_split(train_df, ratio=0.7, seed=seed) # 훈련데이터 split
    test = df[df.kind=='test'] # 테스트 데이터
    train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
    train = pd.concat([train_df,train2]) # 훈련데이터 병합
#     train.shape, valid.shape, test.shape

    x_train = train.drop('answerCode',axis=1)
    y_train = train[['answerCode']]

    x_valid = valid.drop('answerCode',axis=1)
    y_valid = valid[['answerCode']]
    # 사용할 Feature 설정
    FEATS = ['KnowledgeTag',
             'user_correct_answer',
             'user_total_answer',
             'big_category',
             'mid_category',
             'problem_num',
             'month','day','dayname','hour',
             'user_acc',
             'test_mean',
             'test_sum',
             'test_std',
             'tag_std',
             'tag_mean',
             'tag_sum',
             'solvesec'
            ]
    preds = model2.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)

    print(f'시간 초로만 추가 VALID AUC : {auc} ACC : {acc}\n')
#     x_train.shape, y_train.shape, x_valid.shape, y_valid.shape
    FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]
    preds = model4.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)

    print(f'3600 VALID AUC : {auc} ACC : {acc}\n')
    # 사용할 Feature 설정
    FEATS = ['KnowledgeTag',
             'user_correct_answer',
             'user_total_answer',
             'big_category',
             'mid_category',
             'problem_num',
             'month','day','dayname','hour',
             'user_acc',
             'test_mean',
             'test_sum',
             'test_std',
             'tag_std',
             'tag_mean',
             'tag_sum',
             'solvesec',
             'time_category'
            ]
    preds = model5.predict(x_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)

    print(f'category VALID AUC : {auc} ACC : {acc}\n')

seed: 0
시간 초로만 추가 VALID AUC : 0.8279017998383869 ACC : 0.7428861788617886

3600 VALID AUC : 0.8309100018567869 ACC : 0.7479674796747967

category VALID AUC : 0.8295978033901399 ACC : 0.7469512195121951

seed: 1
시간 초로만 추가 VALID AUC : 0.8344024124432114 ACC : 0.7573200992555831

3600 VALID AUC : 0.8343223797187196 ACC : 0.7578163771712159

category VALID AUC : 0.8339913801791548 ACC : 0.75136476426799

seed: 2
시간 초로만 추가 VALID AUC : 0.8213292695968346 ACC : 0.7404276479363501

3600 VALID AUC : 0.8197994272745637 ACC : 0.739433117851815

category VALID AUC : 0.8202113459839906 ACC : 0.739433117851815

seed: 3
시간 초로만 추가 VALID AUC : 0.8380398111330039 ACC : 0.759581881533101

3600 VALID AUC : 0.8372251140774101 ACC : 0.7590841214534594

category VALID AUC : 0.835474803858215 ACC : 0.7605774016923843

seed: 4
시간 초로만 추가 VALID AUC : 0.833747175208205 ACC : 0.7529761904761905

3600 VALID AUC : 0.8349274640876121 ACC : 0.75

category VALID AUC : 0.8316029014364955 ACC : 0.7509920634920635

seed: 

시간 초로만 추가 VALID AUC : 0.8159614779525173 ACC : 0.7378497790868925

3600 VALID AUC : 0.8174671502834718 ACC : 0.7309769268532155

category VALID AUC : 0.8167123837688522 ACC : 0.7349042709867453

seed: 42
시간 초로만 추가 VALID AUC : 0.8229596501689525 ACC : 0.7448928749377179

3600 VALID AUC : 0.8232269926455973 ACC : 0.752366716492277

category VALID AUC : 0.8227767839395747 ACC : 0.7518684603886397

seed: 43
시간 초로만 추가 VALID AUC : 0.8300425251932 ACC : 0.7410938283993979

3600 VALID AUC : 0.8288154099836316 ACC : 0.7375815353738083

category VALID AUC : 0.8277820498071531 ACC : 0.7441043652784747

seed: 44
시간 초로만 추가 VALID AUC : 0.8386049044039119 ACC : 0.7485976542580316

3600 VALID AUC : 0.8389849852365296 ACC : 0.7485976542580316

category VALID AUC : 0.8395001361171025 ACC : 0.7557368689444162

seed: 45
시간 초로만 추가 VALID AUC : 0.827697877209596 ACC : 0.7366071428571429

3600 VALID AUC : 0.8278369633838383 ACC : 0.7371031746031746

category VALID AUC : 0.8263356218434342 ACC : 0.737103174603

시간 초로만 추가 VALID AUC : 0.8335558964618792 ACC : 0.745069033530572

3600 VALID AUC : 0.8330883677892225 ACC : 0.75

category VALID AUC : 0.8325672681228237 ACC : 0.7519723865877712

seed: 83
시간 초로만 추가 VALID AUC : 0.8258995906054729 ACC : 0.7454282964388835

3600 VALID AUC : 0.8278026064537221 ACC : 0.7492781520692974

category VALID AUC : 0.8272648581979211 ACC : 0.7478344562078922

seed: 84
시간 초로만 추가 VALID AUC : 0.8217419125488803 ACC : 0.7340050377833753

3600 VALID AUC : 0.8204611243715403 ACC : 0.7365239294710327

category VALID AUC : 0.8180244781880047 ACC : 0.7360201511335013

seed: 85
시간 초로만 추가 VALID AUC : 0.8259744056250697 ACC : 0.7411764705882353

3600 VALID AUC : 0.8262176945060947 ACC : 0.7436274509803922

category VALID AUC : 0.8247993107135576 ACC : 0.7465686274509804

seed: 86
시간 초로만 추가 VALID AUC : 0.8227189934998808 ACC : 0.7439684884293452

3600 VALID AUC : 0.8234512510450429 ACC : 0.7474150664697193

category VALID AUC : 0.8226821381532368 ACC : 0.7474150664697193

seed

#### seed 13이 적절 
seed: 13
시간 초로만 추가 VALID AUC : 0.8187398163235076 ACC : 0.7406281661600811

3600 VALID AUC : 0.8199340838394311 ACC : 0.7396149949341438

category VALID AUC : 0.8215223259105646 ACC : 0.7406281661600811

In [4]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

model_auc = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)


## 문제푼 시간 초로만 추가한 것

In [5]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model2 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.238892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.582419
[200]	valid_0's binary_logloss: 0.575168
[300]	valid_0's binary_logloss: 0.568893
[400]	valid_0's binary_logloss: 0.563229
[500]	valid_0's binary_logloss: 0.559829
[600]	valid_0's binary_logloss: 0.556637
[700]	valid_0's binary_logloss: 0.553144
[800]	valid_0's binary_logloss: 0.550667
[900]	valid_0's binary_logloss: 0.548314
[1000]	valid_0's binary_logloss: 0.546135
[1100]	valid_0's binary_logloss: 0.542614
[1200]	valid_0's binary_logloss: 0.540631
[1300

### validation

In [169]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]
preds = model2.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8187398163235076 ACC : 0.7406281661600811



## 문제푼 시간 4800초 이상 모두 4800초로 변환

In [8]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model3 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model3.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.067944 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.56096
[200]	valid_0's binary_logloss: 0.554025
[300]	valid_0's binary_logloss: 0.547694
[400]	valid_0's binary_logloss: 0.542811
[500]	valid_0's binary_logloss: 0.536767
[600]	valid_0's binary_logloss: 0.533314
[700]	valid_0's binary_logloss: 0.530218
[800]	valid_0's binary_logloss: 0.527721
[900]	valid_0's binary_logloss: 0.525034
[1000]	valid_0's binary_logloss: 0.522923
[1100]	valid_0's binary_lo

In [48]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec'
        ]
preds = model3.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8302605814985996 ACC : 0.7531645569620253



In [9]:
post_slack("done")

In [19]:
lgbm_predict(test, model3, FEATS, 'LGBM_Baseline_문제푼시간_추가_4800초로수렴.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_4800초로수렴.csv


## 문제푼 시간 3600초 이상 모두 3600으로 변환

In [11]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model4 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

post_slack("done")

lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052752 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3095
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.57308
[200]	valid_0's binary_logloss: 0.564389
[300]	valid_0's binary_logloss: 0.559807
[400]	valid_0's binary_logloss: 0.554239
[500]	valid_0's binary_logloss: 0.550381
[600]	valid_0's binary_logloss: 0.547333
[700]	valid_0's binary_logloss: 0.542663
[800]	valid_0's binary_logloss: 0.539448
[900]	valid_0's binary_logloss: 0.537293
[1000]	valid_0's binary_logloss: 0.533369
[1100]	valid_0's binary_lo

### validation

In [170]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600'
        ]
preds = model4.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8199340838394311 ACC : 0.7396149949341438



In [49]:
lgbm_predict(test, model4, FEATS, 'LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_3600초이상모두3600으로.csv


## 문제푼 시간 카테고리 변수도 추가

In [16]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec',
         'time_category'
        ]

In [17]:
lgb_x_train = lgb.Dataset(x_train[FEATS], y_train)
lgb_x_valid = lgb.Dataset(x_valid[FEATS], y_valid)

model5 = lgb.train(
    params, 
    lgb_x_train,
    valid_sets=[lgb_x_valid],
    verbose_eval=100,
    num_boost_round=3200,
#     early_stopping_rounds=200,
)

preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

[LightGBM] [Info] Number of positive: 1653588, number of negative: 872368
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052055 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3105
[LightGBM] [Info] Number of data points in the train set: 2525956, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639491
[LightGBM] [Info] Start training from score 0.639491
[100]	valid_0's binary_logloss: 0.560899
[200]	valid_0's binary_logloss: 0.552517
[300]	valid_0's binary_logloss: 0.547221
[400]	valid_0's binary_logloss: 0.540192
[500]	valid_0's binary_logloss: 0.535174
[600]	valid_0's binary_logloss: 0.531541
[700]	valid_0's binary_logloss: 0.527031
[800]	valid_0's binary_logloss: 0.523995
[900]	valid_0's binary_logloss: 0.522249
[1000]	valid_0's binary_logloss: 0.519503
[1100]	valid_0's binary_l

### validation

In [172]:
x_train.time_category.value_counts()

4    1008577
5     552061
1     364569
9     362015
3     102175
2      69147
6      27380
7      16761
0      15711
8       7560
Name: time_category, dtype: int64

In [171]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
         'solvesec_3600',
         'time_category'
        ]
preds = model5.predict(x_valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

VALID AUC : 0.8215223259105646 ACC : 0.7406281661600811



In [18]:
post_slack("done")

In [52]:
lgbm_predict(test, model5, FEATS, 'LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv')

writing prediction : output/LGBM_Baseline_문제푼시간_추가_최대3600으로_time_category추가.csv
